In [28]:
#Bibliotecas necessárias
import pandas as pd


#Criação do DataFrame
df = pd.read_csv('desafio_indicium_imdb.csv')

#Remoção das coluna Unnamed, Overview e Series_Title
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.drop(columns=['Overview', 'Series_Title'])

#Remoção das observações com valores ausentes
df = df.dropna()

#Transformação de colunas para tipos numéricos
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')
df['Runtime'] = df['Runtime'].str.replace(' min', '').astype(float)
df['Gross'] = df['Gross'].str.replace(',', '').astype(float)
df['Meta_score'] = pd.to_numeric(df['Meta_score'], errors='coerce')
df['IMDB_Rating'] = pd.to_numeric(df['IMDB_Rating'], errors='coerce')
df['No_of_Votes'] = pd.to_numeric(df['No_of_Votes'], errors='coerce')

In [ ]:
#Criação do modelo

#Bibliotecas necessárias
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer

#Feature selection ( separando as variáveis independentes (X) e dependente (y))
X = df.drop(columns=['IMDB_Rating'])
y = df['IMDB_Rating']

#Identificando as variáveis categóricas e numéricas
var_categoricas = X.select_dtypes(include=['object']).columns
var_numericas = X.select_dtypes(include=[np.number]).columns

#Pré-processamento das variáveis

#Padronização das variáveis numéricas
transf_numerica = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#One-hot encoding
transf_categorica = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#Aplicação as transformações
pre_processo = ColumnTransformer(
    transformers=[
        ('num', transf_numerica, var_numericas),
        ('cat', transf_categorica, var_categoricas)
    ])

#Divisão dos dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Criação do modelo (pipeline que inclui o pré-processamento e o modelo RandomForestRegressor)
modeloRFR = Pipeline(steps=[
    ('preprocessor', pre_processo),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

#Treinamento do modelo
modeloRFR.fit(X_train, y_train)

#Realização das previsões
y_pred = modeloRFR.predict(X_test)

#Cálculo das métricas de performance
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

#Impressão dos resultados
print('Métricas de performance')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R²): {r2}')

In [30]:
#Salvando no formato .pkl
import pickle
arq = 'modeloRFR.pkl'

#Salvando o modelo no arquivo pkl
with open(arq, 'wb') as arquivo:
    pickle.dump(modeloRFR, arquivo)

In [ ]:
#Teste

#Dados de entrada
dados_filme = {
    'Series_Title': ['The Shawshank Redemption'],
    'Released_Year': ['1994'],
    'Certificate': ['A'],
    'Runtime': ['142 min'],
    'Genre': ['Drama'],
    'Overview': ['Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'],
    'Meta_score': [80.0],
    'Director': ['Frank Darabont'],
    'Star1': ['Tim Robbins'],
    'Star2': ['Morgan Freeman'],
    'Star3': ['Bob Gunton'],
    'Star4': ['William Sadler'],
    'No_of_Votes': [2343110],
    'Gross': ['28,341,469']
}

df_filme = pd.DataFrame(dados_filme)

#Aplicando o mesmo pré-processamento aos dados

df_filme = df_filme.drop(columns=['Overview', 'Series_Title'])

df_filme['Released_Year'] = pd.to_numeric(df_filme['Released_Year'], errors='coerce')
df_filme['Runtime'] = df_filme['Runtime'].str.replace(' min', '').astype(float)
df_filme['Gross'] = df_filme['Gross'].str.replace(',', '').astype(float)
df_filme['Meta_score'] = pd.to_numeric(df_filme['Meta_score'], errors='coerce')
df_filme['No_of_Votes'] = pd.to_numeric(df_filme['No_of_Votes'], errors='coerce')

#Identificando as variáveis categóricas e numéricas
var_categoricas = df_filme.select_dtypes(include=['object']).columns
var_numericas = df_filme.select_dtypes(include=[np.number]).columns

transf_numerica = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

transf_categorica = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

pre_processo = ColumnTransformer(
    transformers=[
        ('num', transf_numerica, var_numericas),
        ('cat', transf_categorica, var_categoricas)
    ])

#Carregando o modelo treinado (carregando o pipeline completo incluindo ColumnTransformer ajustado)
import joblib
pipeline_completo = joblib.load('modeloRFR.pkl')

#Realização das previsões
avaliacao_pred = pipeline_completo.predict(df_filme)

# Imprimir as previsões
print(f'Nota IMDb prevista: {round(avaliacao_pred[0], 1)}')
